# Traffic data analysis

The objective of this notebook is to use data provided by Victor Couture and team to crown "Canada's Worst Commute".

In [1]:
import pandas as pd

In [ ]:
# In kilometres.
MIN_TRIP_DISTANCE = 10

# Local hour when rush hour starts.
MORNING_RUSH_HOUR_START = 8
MORNING_RUSH_HOUR_END = 10
EVENING_RUSH_HOUR_START = 17
EVENING_RUSH_HOUR_END = 19

MIN_DATA_POINTS_NONRUSHHOUR = 10
MIN_DATA_POINTS_RUSHHOUR = 5

START_DATE = pd.Timestamp(f"2024-10-01")
END_DATE = pd.Timestamp(f"2025-09-30")

Filtering for only the columsn I need so the file doesn't get too large.

In [3]:
df = pd.read_parquet('data/victor/alltrips_canada_2024_2025.parquet')

# Create a datetime column for analysis
df.head(1)

,tripid,mode,citycode,cityname_corrected,time_full_str,traffic_min,notraffic_min,dayofweek,tz,trip_dist,lat_dest,lon_dest,lat_orig,lon_orig
0,101497520,0.0,20370,Calgary,20240111 00:56:12,13.3,13.1,3,America/Edmonton,9.548,51.060974,-113.951439,51.04414,-114.059586


In [4]:
df["cityname_corrected"] = df["cityname_corrected"].str.replace("é", "e")
df["cityname_corrected"] = df["cityname_corrected"].replace("Kitchener-Cambrigde-Waterloo", "Kitchener-Cambridge-Waterloo")

In [5]:
# Create a datetime column for analysis
df['time_dt'] = pd.to_datetime(df['time_full_str'])

In [6]:
df = df[(df['time_dt'] <= END_DATE) & (df['time_dt'] >= START_DATE)]

df.head()

,tripid,mode,citycode,cityname_corrected,time_full_str,traffic_min,notraffic_min,dayofweek,tz,trip_dist,lat_dest,lon_dest,lat_orig,lon_orig,time_dt
3970387,101497520,0.0,20370,Calgary,20241101 09:05:27,15.650000,15.283333,4,America/Edmonton,9.603,51.060974,-113.951439,51.044140,-114.059586,2024-11-01 09:05:27
3970388,101497521,0.0,20370,Calgary,20241031 00:02:12,13.583333,15.983333,3,America/Edmonton,9.947,51.044140,-114.059586,51.060974,-113.951439,2024-10-31 00:02:12
3970389,101497521,0.0,20370,Calgary,20241031 08:15:44,21.383333,15.983333,3,America/Edmonton,9.947,51.044140,-114.059586,51.060974,-113.951439,2024-10-31 08:15:44
3970390,101497540,0.0,20370,Calgary,20241101 14:01:34,23.166667,19.516667,4,America/Edmonton,15.527,50.949066,-114.085335,51.044357,-114.061646,2024-11-01 14:01:34
3970391,101497551,0.0,20370,Calgary,20241030 20:16:30,13.583333,12.516667,2,America/Edmonton,10.061,51.044140,-114.063362,51.000732,-114.118637,2024-10-30 20:16:30


Check data start time and end time.

In [7]:
display(df["time_full_str"].min(), df["time_full_str"].max())

'20241029 05:39:11'

'20250929 23:59:46'

How many cities is there data for?

In [8]:
df["cityname_corrected"].unique()

array(['Calgary', 'Edmonton', 'Halifax', 'Hamilton',
       'Kitchener-Cambridge-Waterloo', 'London', 'Montreal', 'Oshawa',
       'Ottawa', 'Quebec', 'Saskatoon', 'St. Catharines-Niagara',
       'Toronto', 'Vancouver', 'Victoria', 'Windsor', 'Winnipeg'],
      dtype=object)

Let's now see how many cities we have in this dataset.

In [9]:
df.loc[:, ["cityname_corrected", "tz"]].drop_duplicates()

,cityname_corrected,tz
3970387,Calgary,America/Edmonton
3971133,Edmonton,America/Edmonton
3971843,Halifax,America/Halifax
3972826,Hamilton,America/Toronto
3973695,Kitchener-Cambridge-Waterloo,America/Toronto
3974686,London,America/Toronto
3975805,Montreal,America/Toronto
3977878,Oshawa,America/Toronto
3978757,Ottawa,America/Toronto
3980304,Quebec,America/Toronto


For this table, I’ve excluded routes that are less than 10km long, and only included routes with at least 3 data points collected over the past 6 months in both non-rush hour and rush hour (8-10am, 5-7pm) times. I've excluded everything under 10km, because that doesn't really feel like it qualifies as a commute.

In [10]:
# --- Rush Hour Analysis most recent year
weekdays_df = df[(df['time_dt'].dt.weekday < 5) & (df['trip_dist'] >= MIN_TRIP_DISTANCE) & (df['mode'] == 0)].copy()

hour = weekdays_df['time_dt'].dt.hour

is_rush_hour = ((hour >= MORNING_RUSH_HOUR_START) & (hour < MORNING_RUSH_HOUR_END)) | ((hour >= EVENING_RUSH_HOUR_START) & (hour < EVENING_RUSH_HOUR_END))

weekdays_df['period'] = 'Non-Rush Hour'
weekdays_df.loc[is_rush_hour, 'period'] = 'Rush Hour'

In [ ]:
# --- Rush Hour Analysis most recent year

# Ensure origin/dest exist (from earlier cells)
if 'origin' not in weekdays_df.columns and 'lat_orig' in weekdays_df.columns:
    weekdays_df['origin'] = weekdays_df['lat_orig'].astype(str) + ',' + weekdays_df['lon_orig'].astype(str)
if 'dest' not in weekdays_df.columns and 'lat_dest' in weekdays_df.columns:
    weekdays_df['dest'] = weekdays_df['lat_dest'].astype(str) + ',' + weekdays_df['lon_dest'].astype(str)

# Group traffic_min by trip/period to get mean and count per period
grouping_cols = ['tripid', 'cityname_corrected', 'trip_dist', 'period']
agg_times = weekdays_df.groupby(grouping_cols)['traffic_min'].agg(['mean', 'count']).reset_index()

# Pivot traffic stats so we have mean/count per period side-by-side
comparison_table = agg_times.pivot_table(
    index=[col for col in grouping_cols if col != 'period'],
    columns='period',
    values=['mean', 'count']
).reset_index()

# Flatten multiindex column names
comparison_table.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in comparison_table.columns.values]

# Rename traffic columns to friendly names
comparison_table.rename(columns={
    'mean_Non-Rush Hour': 'mean_non_rush_hour',
    'mean_Rush Hour': 'mean_rush_hour',
    'count_Non-Rush Hour': 'count_non_rush_hour',
    'count_Rush Hour': 'count_rush_hour',
    'tripid_': 'tripid',
    'mode_': 'mode',
    'cityname_corrected_': 'cityname_corrected',
    'trip_dist_': 'trip_dist',
}, inplace=True)

# Compute a single overall mean for notraffic_min (include all periods)
notraffic_overall = weekdays_df.groupby(['tripid', 'cityname_corrected', 'trip_dist'])['notraffic_min'].mean().reset_index().rename(columns={'notraffic_min': 'mean_notraffic'})

# Compute 90th percentile travel time during Rush Hour per trip (p90)
rush_df = weekdays_df[weekdays_df['period'] == 'Rush Hour']
p90_rush = rush_df.groupby(['tripid', 'cityname_corrected', 'trip_dist'])['traffic_min'].quantile(0.9).reset_index().rename(columns={'traffic_min': 'p90_rush_hour'})

# Merge the overall notraffic mean and p90 into the comparison table
comparison_table = comparison_table.merge(notraffic_overall, on=['tripid', 'cityname_corrected', 'trip_dist'], how='left')
comparison_table = comparison_table.merge(p90_rush, on=['tripid', 'cityname_corrected', 'trip_dist'], how='left')

# Bring origin/dest into the comparison table using a representative value (first)
orig_dest = weekdays_df.groupby(['tripid', 'cityname_corrected', 'trip_dist']).agg({'origin': 'first', 'dest': 'first'}).reset_index()
comparison_table = comparison_table.merge(orig_dest, on=['tripid', 'cityname_corrected', 'trip_dist'], how='left')

comparison_table.fillna(0, inplace=True)
# New metric: difference between rush hour mean and overall notraffic (free-flow)
comparison_table['rush_vs_freeflow_diff'] = comparison_table['mean_rush_hour'] - comparison_table['mean_notraffic']
# Percent impact relative to free-flow (notraffic)
# Avoid division by zero
comparison_table['rush_vs_freeflow_pct'] = comparison_table.apply(lambda r: (r['rush_vs_freeflow_diff'] / r['mean_notraffic'] * 100) if r['mean_notraffic'] else 0, axis=1)
comparison_table['p90_vs_freeflow'] = comparison_table['p90_rush_hour'] - comparison_table['mean_notraffic']

# Convert count columns to integers where present
for col in ['count_non_rush_hour', 'count_rush_hour']:
    if col in comparison_table.columns:
        comparison_table[col] = comparison_table[col].astype(int)

# Round numeric columns: trip_dist to 1 decimal, others to 2 decimals
if 'trip_dist' in comparison_table.columns:
    comparison_table['trip_dist'] = comparison_table['trip_dist'].round(1)
num_cols = ['mean_non_rush_hour', 'mean_rush_hour', 'mean_notraffic', 'rush_vs_freeflow_diff', 'rush_vs_freeflow_pct', 'p90_rush_hour']
for c in num_cols:
    if c in comparison_table.columns:
        comparison_table[c] = comparison_table[c].round(2)

# Final sorting and filtering
final_table = comparison_table.sort_values('rush_vs_freeflow_diff', ascending=False)
final_table = final_table[(final_table['count_non_rush_hour'] >= MIN_DATA_POINTS_NONRUSHHOUR) & (final_table['count_rush_hour'] >= MIN_DATA_POINTS_RUSHHOUR)]

# Include origin/dest and p90 in display
final_table["rank"] = range(1,len(final_table)+1)
display_cols = ['tripid', 'cityname_corrected', 'origin', 'dest', 'trip_dist', 'mean_notraffic', 'mean_rush_hour', 'rush_vs_freeflow_diff', 'rush_vs_freeflow_pct', 'p90_rush_hour', "p90_vs_freeflow"]
final_table = final_table.set_index('rank')
final_table[display_cols].to_csv("results-victor.csv")
final_table[display_cols].head(10)

,tripid,cityname_corrected,origin,dest,trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_rush_hour,p90_vs_freeflow
rank,,,,,,,,,,,
1,101617601,Toronto,"43.867138,-79.50793","43.65489,-79.385704",43.0,35.78,62.09,26.31,73.53,69.35,33.564375
2,101610311,Toronto,"43.821083,-79.34588","43.652157,-79.380554",22.8,18.95,40.18,21.22,111.97,51.27,32.312698
3,303928020,Montreal,"45.50147,-73.46785","45.694843,-73.66869",38.5,31.76,51.82,20.06,63.17,67.24,35.481270
4,304101661,Toronto,"43.401985,-79.76679","43.648182,-79.45746",41.1,26.67,45.53,18.86,70.71,54.93,28.255556
5,101636931,Vancouver,"49.299095,-122.79921","49.240627,-123.11095",33.2,35.46,52.34,16.88,47.59,55.80,20.332941
6,304156470,Vancouver,"49.11764,-122.86204","49.22628,-122.941",19.8,21.22,36.23,15.01,70.71,39.31,18.092222
7,14182640,Vancouver,"49.25273,-123.11198","49.236145,-122.90907",18.7,21.92,36.14,14.22,64.86,51.09,29.168651
8,431073181,Montreal,"45.53017,-73.60811","45.704674,-73.618225",29.5,27.10,40.76,13.67,50.43,54.03,26.932083
9,303925231,Montreal,"45.483418,-73.721565","45.472343,-73.61273",10.6,9.62,23.25,13.63,141.72,26.88,17.257917


## Mapping results

In [12]:
# # Save top-3 route maps per city and fetch fresh geometry from Google Directions for each origin/destination
# import os
# import pathlib
# import re
# import time
# import requests
# import polyline
# import matplotlib.pyplot as plt
# import geopandas as gpd
# from shapely.geometry import LineString

# out_dir = pathlib.Path('plots_by_route')
# out_dir.mkdir(parents=True, exist_ok=True)

# # Read API key (file fallback then env)
# API_KEY = None
# key_path = os.path.join(os.getcwd(), 'google_maps_api_key.txt')
# if os.path.exists(key_path):
#     with open(key_path, 'r', encoding='utf-8') as fh:
#         API_KEY = fh.read().strip()
# if not API_KEY:
#     API_KEY = os.getenv('GOOGLE_API_KEY')
# if not API_KEY:
#     raise RuntimeError('Google Maps API key not found. Add google_maps_api_key.txt or set GOOGLE_API_KEY')

# def safe(s):
#     if s is None:
#         return 'unknown'
#     s = str(s)
#     s = re.sub(r'[^A-Za-z0-9_\-]', '_', s)
#     return s[:180]

# if 'final_table' not in globals():
#     raise RuntimeError('`final_table` not found. Run the Rush Hour Analysis cell first.')

# ft = final_table.reset_index()
# if 'rank' not in ft.columns:
#     ft.insert(0, 'rank', range(1, len(ft) + 1))

# # cache to avoid duplicate API calls for identical origin/dest within this run
# route_cache = {}

# def fetch_route_geometry(origin_str, dest_str, max_retries=3):
#     key = f'{origin_str}|{dest_str}'
#     if key in route_cache:
#         return route_cache[key]
#     url = 'https://maps.googleapis.com/maps/api/directions/json'
#     params = {
#         'origin': origin_str,
#         'destination': dest_str,
#         'alternatives': 'false',
#         'mode': 'driving',
#         'key': API_KEY
#     }
#     backoff = 1.0
#     for attempt in range(1, max_retries + 1):
#         try:
#             resp = requests.get(url, params=params, timeout=15)
#             resp.raise_for_status()
#             j = resp.json()
#         except Exception as e:
#             if attempt == max_retries:
#                 raise
#             time.sleep(backoff)
#             backoff *= 2
#             continue
#         status = j.get('status')
#         if status != 'OK' or not j.get('routes'):
#             # For non-retryable API errors, return None
#             return None
#         ov = j['routes'][0].get('overview_polyline', {}).get('points')
#         if not ov:
#             return None
#         coords_latlon = polyline.decode(ov)  # list of (lat, lon)
#         coords = [(lon, lat) for lat, lon in coords_latlon]
#         ls = LineString(coords)
#         # create polygon buffer of 100 m by projecting to web mercator
#         single_line_gdf = gpd.GeoDataFrame([{'tripid': None, 'origin': origin_str, 'dest': dest_str, 'linestring': ls}], geometry='linestring', crs='EPSG:4326')
#         web = single_line_gdf.to_crs(epsg=3857)
#         web['polygon_100m'] = web.geometry.buffer(100)
#         poly = gpd.GeoDataFrame(web.copy(), geometry=web['polygon_100m'], crs=web.crs).to_crs(epsg=4326)
#         route_cache[key] = (ls, poly)
#         return ls, poly
#     return None

# for city in ft['cityname_corrected'].unique():
#     city_rows = ft[ft['cityname_corrected'] == city].sort_values('rush_vs_freeflow_diff', ascending=False).head(3)
#     if city_rows.empty:
#         continue
#     for _, crow in city_rows.iterrows():
#         rank = int(crow.get('rank', -1))
#         tripid = crow.get('tripid')
#         origin_val = crow.get('origin')
#         dest_val = crow.get('dest')
#         if not origin_val or not dest_val:
#             print(f'Missing origin/dest for trip {tripid} (rank {rank}) in {city}; skipping')
#             continue
#         # origin_val/dest_val should be 'lat,lon' strings; ensure trimmed
#         origin_str = str(origin_val).strip()
#         dest_str = str(dest_val).strip()
#         try:
#             res = fetch_route_geometry(origin_str, dest_str)
#         except Exception as e:
#             print(f'Error fetching route for {tripid} in {city}: {e}; skipping')
#             continue
#         if not res:
#             print(f'No route geometry returned for {tripid} in {city}; skipping')
#             continue
#         ls, poly = res
#         fname = out_dir / f"{rank:03d}_{safe(tripid)}_{safe(city)}.png"
#         fig, ax = plt.subplots(1, 1, figsize=(8, 6))
#         try:
#             if poly is not None and not poly.empty:
#                 poly.to_crs(epsg=3857).plot(ax=ax, color='orange', alpha=0.4, edgecolor='darkorange')
#             single_line = gpd.GeoDataFrame([{'linestring': ls}], geometry='linestring', crs='EPSG:4326')
#             single_line.to_crs(epsg=3857).plot(ax=ax, color='red', linewidth=3)
#             minx, miny, maxx, maxy = single_line.to_crs(epsg=3857).total_bounds
#             buf = max((maxx - minx), (maxy - miny)) * 0.25 if maxx > minx and maxy > miny else 200
#             ax.set_xlim(minx - buf, maxx + buf)
#             ax.set_ylim(miny - buf, maxy + buf)
#             try:
#                 import contextily as ctx
#                 ctx.add_basemap(ax)
#             except Exception:
#                 pass
#             ax.set_axis_off()
#             ax.set_title(f'Rank {rank} — Trip {tripid} — {city}')
#             fig.savefig(fname, dpi=150, bbox_inches='tight')
#             plt.close(fig)
#             print(f'Saved {fname}')
#         except Exception as e:
#             print(f'Failed to save route {tripid} (rank {rank}) for city {city}: {e}')
